## First we initialize our globals

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [3]:
# add project dir to python path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    import sys, os
    sys.path.append("/Workspace" + os.path.dirname(os.path.dirname(notebook_path)))
except:
    pass

In [4]:
# load table into spark dataframe
cows_bff = spark.read.table("farm.db.cows_bff")

## Some graphs and charts

In [5]:
display(cows_bff.limit(10))

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:567: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not is_datetime64tz_dtype(pser.dtype):
/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:386: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):


,cow_name,meal_start,meal_end,meal_start_time,meal_end_time,duration,date,day
0,Butterscotch,32171,34719,8:56:11,9:38:39,2548,2023-05-20,Saturday
1,Hershey,34538,34757,9:35:38,9:39:17,219,2023-05-20,Saturday
2,Mocha,33395,34697,9:16:35,9:38:17,1302,2023-05-20,Saturday
3,Cocoa,33395,34697,9:16:35,9:38:17,1302,2023-05-20,Saturday
4,Nutella,31874,33348,8:51:14,9:15:48,1474,2023-05-20,Saturday
5,Brandy,32597,34302,9:03:17,9:31:42,1705,2023-05-20,Saturday
6,Peaches,33373,33816,9:16:13,9:23:36,443,2023-05-20,Saturday
7,Marshmallow,32569,34581,9:02:49,9:36:21,2012,2023-05-20,Saturday
8,Popcorn,33251,34757,9:14:11,9:39:17,1506,2023-05-20,Saturday
9,Muffin,32041,33446,8:54:01,9:17:26,1405,2023-05-20,Saturday


## Lets compute meal overlap

In [8]:
from cow_bff.heatmap import compute_heatmap

df = compute_heatmap(cows_bff)

display(df.limit(10))

+------+------------+------------------+
|  cow1|        cow2|          distance|
+------+------------+------------------+
|Brandy|   Buttercup|             536.0|
|Brandy|Butterscotch| 533.9230769230769|
|Brandy|       Cocoa| 519.3846153846154|
|Brandy|       Daisy|511.15384615384613|
|Brandy|      Dottie| 318.7692307692308|
+------+------------+------------------+
only showing top 5 rows



NameError: name 'lit' is not defined

# Display Heatmap

In [7]:
pdf = df.toPandas()
pdf = pdf.pivot(index='cow1', columns='cow2', values='distance').fillna(0)

import plotly.express as px
fig = px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"), title="Cow BFFs", color_continuous_scale='redor')
px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"),)

fig.update_layout(width=800,height=500)
fig.show()
